# 문제해결
### 취준생을 위한 서비스를 만들여고 합니다.
- 입력 : 사용자들이 입력 내용
    - 직군(job_role)
    - 숙련도(level)
- 출력 : json 형태
    ```
    직군명(job_role)
    숙련도 수준(level)
    핵심 기술 역량(core_skills)
    필수 도구 및 기술 스택(tools)
    비기술 역량(soft_skills)
    ```

## 방법1

### API Key 로딩

In [2]:
# API key 로딩
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ["OPENAI_API_KEY"]


# 라이브러리 로딩

In [5]:
# 라이브러리
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

# llm 객체 생성

In [10]:
llm = ChatOpenAI(model="gpt-5-nano", api_key=api_key)
# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=api_key)

## Output 파서 객체 생성

In [11]:
parser = JsonOutputParser()

# 프롬프트 템플릿 생성

- JsonOutputParser() 기본

In [12]:


prompt_template = PromptTemplate(
    template="""
    현재 IT, AI 기술을 반영한 직군 정보를 제공해주세요.\n{format_instructions}\n{job_role}, {level}\n
    출력 형식을 다음과 같이 json으로 해주세요.
    [출력 형식]
    직군명: 
    숙련도 수준:
    핵심 기술 역량:
    필수 도구 및 기술 스택:
    비기술 역량:
    직업 상세 설명:
    """,
    input_variables=["job_role", "level"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt_template

PromptTemplate(input_variables=['job_role', 'level'], input_types={}, partial_variables={'format_instructions': 'Return a JSON object.'}, template='\n    현재 IT, AI 기술을 반영한 직군 정보를 제공해주세요.\n{format_instructions}\n{job_role}, {level}\n\n    출력 형식을 다음과 같이 json으로 해주세요.\n    [출력 형식]\n    직군명: \n    숙련도 수준:\n    핵심 기술 역량:\n    필수 도구 및 기술 스택:\n    비기술 역량:\n    직업 상세 설명:\n    ')

In [13]:
# job_role과 level 매개변수 넘김
prompt = prompt_template.format(job_role="소프트웨어 엔지니어", level="초급")
print(prompt)


    현재 IT, AI 기술을 반영한 직군 정보를 제공해주세요.
Return a JSON object.
소프트웨어 엔지니어, 초급

    출력 형식을 다음과 같이 json으로 해주세요.
    [출력 형식]
    직군명: 
    숙련도 수준:
    핵심 기술 역량:
    필수 도구 및 기술 스택:
    비기술 역량:
    직업 상세 설명:
    


# llm 응답 요청하기

In [14]:
response = llm.invoke(prompt)

# 응답 파싱하기

In [58]:
print(response.content)

{
  "직군명": "소프트웨어 엔지니어",
  "숙련도 수준": "초급",
  "핵심 기술 역량": "Python, JavaScript/TypeScript, Java 등 주요 프로그래밍 언어; 자료구조/알고리즘 기본; RESTful API 설계 및 사용; 데이터베이스 기본(SQL/NoSQL); 소프트웨어 개발 생명주기(SDLC) 이해; 테스트 주도 개발(TDD)/유닛 테스트; 기본 클라우드 이해(배포/인프라 기초); AI 서비스 연동 이해(클라우드 AI 서비스/API 사용); 버전 관리(Git) 및 협업 도구",
  "필수 도구 및 기술 스택": "Git, GitHub/GitLab, Linux/macOS 쉘, VS Code/IDE, Docker, REST API, SQL(PostgreSQL/MySQL) 및 NoSQL(MongoDB) DB, 테스트 프레임워크(JUnit, PyTest 등), 빌드 도구(Maven/Gradle, npm), CI/CD(예: GitHub Actions), 기본 AWS/Azure/GCP 서비스, API 문서화(Swagger/OpenAPI), 간단한 Kubernetes/Docker Compose 기본, AI API(OpenAI, AWS SageMaker 엔드포인트) 사용 경험",
  "비기술 역량": "문제 해결 및 분석 능력, 빠른 학습 의지, 의사소통 및 협업, 코드 리뷰에 적극 참여, 문서화 습관, 시간 관리, 피드백 수용, 애자일/스크럼 이해, 사용자 관점의 사고",
  "직업 상세 설명": "소프트웨어 엔지니어 초급으로서 요구사항 분석 및 명세에 맞춰 작고 유지보수 가능한 기능을 구현하고 버그를 수정합니다. 코드 작성 시 가독성과 유지보수를 우선으로 하며 단위 테스트와 문서화를 통해 품질을 확보합니다. 팀의 애자일 프로세스에 따라 작업을 계획하고 PR 코드 리뷰에 참여합니다. AI 기능이 포함된 애플리케이션의 경우 AI API를 활용한 기능 연동, 데이터 흐름 설계, 모델 호출 빈도 및 성능 모니터링에 참여합니다. 데이터베이스 

# LCEL 문법으로 chain 객체로 요청하기

In [59]:
# chain = 프롬프트템플릿 | llm_모델 | 파서 
# chain.invoke({"변수": "질문"})
chain = prompt_template | llm | parser

email = "AI 서비스 기획자"
level = "초급"

response = chain.invoke({"job_role": email, "level": level})

In [60]:
response

{'직군명': 'AI 서비스 기획자',
 '숙련도 수준': '초급',
 '핵심 기술 역량': ['AI 서비스 문제 정의 및 가설 설정',
  '데이터 기반 의사결정 및 지표 설계',
  'ML/AI 기본 지식 이해(모델 유형, 평가 지표, 데이터 처리와 파이프라인의 기초)',
  '프롬프트 엔지니어링의 기초 및 LLM 활용 시나리오 설계',
  '엔드-투-엔드 AI 서비스 로드맵 구성 및 백로그 관리',
  '사용자 연구 및 요구사항 도출(인터뷰, 설문, 사용자 여정 맵)',
  '윤리, 데이터 프라이버시 및 보안 고려',
  '스토리텔링 및 커뮤니케이션 능력'],
 '필수 도구 및 기술 스택': ['Jira, Confluence 또는 Notion 같은 협업 도구',
  'Figma',
  'SQL, Excel/Sheets, Power BI 또는 Tableau',
  'OpenAI API, Azure OpenAI, Google Vertex AI 등의 기본 활용',
  'A/B 테스트 도구: Optimizely, Google Optimize',
  '데이터 파이프라인 기초 지식: ETL 개념',
  '문서화 및 커뮤니케이션: Markdown, Slack',
  '기초 프로그래밍 지식: Python 기초 및 API 호출'],
 '비기술 역량': ['커뮤니케이션 및 스토리텔링',
  '이해관계자 관리 및 협업',
  '문제 해결 및 비즈니스 관점의 의사결정',
  '사용자 중심 사고 및 공감 능력',
  '우선순위 설정 및 일정 관리',
  '학습 민첩성과 피드백 수용'],
 '직업 상세 설명': 'AI 서비스 기획자는 AI 기반 서비스의 전략 수립부터 실행까지의 초기 단계에 참여합니다. 주요 임무는 사용자 문제를 AI 솔루션으로 정의하고, 가설을 세워 KPI를 설계하며, 데이터와 실험을 통해 가치를 검증하는 것입니다. 담당자는 제품 관리자, 데이터 사이언티스트, 엔지니어, 디자이너와 협업하여 MVP 로드맵을 작성하고 백로그를 관리합니다. 실무에서의 하루는 고객 인

## 방법2

- 응답 객체 생성

In [62]:
# BaseModel 상속 받기
class JobRoleParser(BaseModel):
    job_role: str = Field(description="직군명")
    level: str = Field(description="숙련도 수준")
    core_skills: str = Field(description="핵심 기술 역량")
    tools: str = Field(description="필수 도구 및 기술 스택")
    soft_skills: str = Field(description="비기술 역량")
    explain: str = Field(description="직업 설명")

- outputparser 객체 생성

In [63]:
parser = JsonOutputParser(pydantic_object=JobRoleParser)
parser

JsonOutputParser(pydantic_object=<class '__main__.JobRoleParser'>)

- format 명령 확인

In [64]:
print(parser.get_format_instructions())

STRICT OUTPUT FORMAT:
- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.
- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).
- Do not prepend or append any text (e.g., do not write "Here is the JSON:").
- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema (shown in a code block for readability only — do not include any backticks or Markdown in your output):


- 템플릿 객체 생성

In [65]:
prompt_template = PromptTemplate(
    template="현재 IT, AI 기술을 반영한 직군 정보를 제공해주세요.\n{format_instructions}\n{job_role}, {level}\n",
    input_variables=["job_role", "level"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)
prompt_template

PromptTemplate(input_variables=['job_role', 'level'], input_types={}, partial_variables={'format_instructions': 'STRICT OUTPUT FORMAT:\n- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.\n- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).\n- Do not prepend or append any text (e.g., do not write "Here is the JSON:").\n- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.\n\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is

In [66]:
# job_role과 level 매개변수 넘김
job_role = "IT/AI 컨설턴트"
level = "초급"
prompt = prompt_template.format(job_role=job_role, level=level)
print(prompt)

현재 IT, AI 기술을 반영한 직군 정보를 제공해주세요.
STRICT OUTPUT FORMAT:
- Return only the JSON value that conforms to the schema. Do not include any additional text, explanations, headings, or separators.
- Do not wrap the JSON in Markdown or code fences (no ``` or ```json).
- Do not prepend or append any text (e.g., do not write "Here is the JSON:").
- The response must be a single top-level JSON value exactly as required by the schema (object/array/etc.), with no trailing commas or comments.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]} the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema (shown in a code block for readability only — do not include any backti

# 프롬프트 템플릿 실행 요청

In [67]:
# llm에 prompt 보내기
response = llm.invoke(prompt)

# 응답 받은 결과를 parser로 파싱하기
parsed_output = parser.invoke(response)
print(parsed_output)

{'job_role': 'IT/AI 컨설턴트', 'level': '초급', 'core_skills': 'IT/AI 도메인 기본 지식, 문제 정의 및 요구사항 분석, 데이터 이해 및 해석 능력, 기본 분석 및 문제 해결 방법론, 프로젝트 관리의 기초', 'tools': 'Python 기초, SQL 기초, Excel 고급 사용, 데이터 시각화 도구 기초(Tableau/Power BI), 협업 및 버전 관리 도구(Jira/Confluence, Git 기초)', 'soft_skills': '의사소통 능력, 팀워크, 적극적 학습 태도, 고객 요구사항 파악 능력, 시간 관리 및 우선순위 설정', 'explain': 'IT/AI 컨설턴트로서 기업의 비즈니스 문제를 IT/AI 관점에서 분석하고 솔루션 설계 및 도입을 지원합니다. 초급은 데이터 이해, 요구사항 수집, 기본 분석, 도구 사용의 기초를 학습하고 고객과의 소통 및 제안서 작성의 기초를 익히는 것을 목표로 합니다.'}


In [69]:
import json
from rich.console import Console

console = Console()

json_data = json.dumps(parsed_output, indent=2, ensure_ascii=False)
console.print(json_data)

{
  "job_role": "IT/AI 컨설턴트",
  "level": "초급",
  "core_skills": "IT/AI 도메인 기본 지식, 문제 정의 및 요구사항 분석, 데이터 이해 및 해석 능력, 기본 분석 및 문제 
해결 방법론, 프로젝트 관리의 기초",
  "tools": "Python 기초, SQL 기초, Excel 고급 사용, 데이터 시각화 도구 기초(Tableau/Power BI), 협업 및 버전 관리 
도구(Jira/Confluence, Git 기초)",
  "soft_skills": "의사소통 능력, 팀워크, 적극적 학습 태도, 고객 요구사항 파악 능력, 시간 관리 및 우선순위 설정",
  "explain": "IT/AI 컨설턴트로서 기업의 비즈니스 문제를 IT/AI 관점에서 분석하고 솔루션 설계 및 도입을 지원합니다. 
초급은 데이터 이해, 요구사항 수집, 기본 분석, 도구 사용의 기초를 학습하고 고객과의 소통 및 제안서 작성의 기초를 
익히는 것을 목표로 합니다."
}

# LCEL로 실행
- LCEL(LangChain Expression Language)

In [73]:
# chain = 프롬프트템플릿 | llm_모델 | 파서 
# chain.invoke({"변수1": "질문1", "변수2": "질문2",})
chain = prompt_template | llm | parser

job_role = "2026년 IT/AI 사업팀 매니저"
level = "초급"

response = chain.invoke({"job_role":job_role, "level":level})

In [74]:
response

{'job_role': '2026년 IT/AI 사업팀 매니저',
 'level': '초급',
 'core_skills': 'AI/데이터 중심의 비즈니스 요구분석, 프로젝트 관리 및 로드맵 수립, 이해관계자 관리, 기본 AI/ML 개념과 데이터 파이프라인 흐름 이해, 요구사항의 우선순위화 및 MVP 정의, 데이터 품질 및 효과성 평가를 위한 KPI 설계',
 'tools': 'Jira/Confluence를 활용한 작업 관리, MS Project 또는 Jira를 통한 로드맵 관리, 데이터 시각화 도구( Tableau, Power BI ), 클라우드 기본 활용(AWS/Azure/GCP), SQL 기본 쿼리 작성 및 간단한 Python/R 스크립트 이해, Git 및 협업 워크플로우, Slack/Teams를 통한 커뮤니케이션',
 'soft_skills': '의사소통과 협업, 이해관계자 관리 및 설득력, 문제 해결 및 의사결정, 리더십과 팀 멘토링, 우선순위 설정과 시간 관리, 변화 관리에 대한 유연성',
 'explain': 'IT/AI 사업팀의 초급 매니저로서 비즈니스 요구를 분석하고 AI 프로젝트의 초기 설계와 실행을 주도합니다. 데이터 기반 의사결정을 촉진하고 이해관계자와의 원활한 소통으로 요구사항을 구체화한 후, 로드맵을 수립하고 MVP를 빠르게 출시합니다. 팀 간 협업을 통해 프로젝트를 추적하고 성과를 KPI로 측정하며, ROI 및 고객 가치 창출의 근거를 제시합니다.'}

In [75]:
# chain = 프롬프트템플릿 | llm_모델 | 파서 
# chain.invoke({"변수1": "질문1", "변수2": "질문2",})
chain = prompt_template | llm | parser

job_role = "2026년 데이터 사이언티스트"
level = "초급"

response = chain.invoke({"job_role":job_role, "level":level})

In [76]:
response

{'job_role': '데이터 사이언티스트',
 'level': '초급',
 'core_skills': '통계학 기초 이해, 데이터 탐색적 분석(EDA), 데이터 정제 및 품질 관리, 특징 엔지니어링 및 피처링 기본, 지도학습 및 비지도학습 알고리즘의 기초 이해(선형/로지스틱 회귀, 결정 트리, 랜덤 포레스트, K-최근접 이웃, SVM, 군집화), 모델 평가 및 검증(교차검증, 성능 지표 이해), 파이프라인 구축 및 모델 서빙의 기초 이해, SQL 및 데이터베이스 쿼리 능력, 파이썬 데이터 과학 생태계(pandas, NumPy, matplotlib/seaborn), 머신러닝 프레임워크 기본(TensorFlow/PyTorch) 기초, 데이터 시각화 및 스토리텔링, 데이터 파이프라인 이해',
 'tools': 'Python, SQL, R(선택), Jupyter Notebook, VS Code, Git/GitHub, pandas, NumPy, scikit-learn, matplotlib/seaborn, TensorFlow 또는 PyTorch 기초, MLflow/MLops 기본, 데이터베이스(MySQL, PostgreSQL), Tableau/Power BI 기초, 클라우드 서비스(AWS/GCP/Azure) 기초, GitHub Actions',
 'soft_skills': '커뮤니케이션 및 협업 능력, 문제 해결 및 분석적 사고, 문서화와 기술적 소통, 피드백 수용 및 반영, 자가주도 학습 및 시간 관리, 윤리적 데이터 처리 및 개인정보 보호 의식, 이해관계자 관리 및 프레젠테이션 능력',
 'explain': '2026년 기준 데이터 과학 분야의 초급 직무로, 데이터 수집/정제에서부터 탐색적 분석, 특징 엔지니어링, 간단한 감독 학습 모델의 개발과 평가, 모델의 해석 가능성 및 재현성 확보를 담당합니다. 주로 데이터 엔지니어, 데이터 애널리스트와 협업하며, 비즈니스 문제를 이해하고 데이터 기반 의사결정을 지원합니다. 배포 초기 단계의 모델을 프로토타이핑하고, ML 

In [77]:
# chain = 프롬프트템플릿 | llm_모델 | 파서 
# chain.invoke({"변수1": "질문1", "변수2": "질문2",})
chain = prompt_template | llm | parser

job_role = "2026년 AI기반 회계사"
level = "초급"

response = chain.invoke({"job_role":job_role, "level":level})

In [78]:
response

{'job_role': 'AI 기반 회계사',
 'level': '초급',
 'core_skills': '데이터 분석 및 해석, 회계 원리 및 감사 지식, 머신러닝/딥러닝 기초, 재무 모델링, 데이터 정제 및 시각화',
 'tools': 'Python(Pandas/Numpy/Scikit-learn), SQL, Excel/Power Query, ERP 시스템(SAP/Oracle), 회계 소프트웨어, RPA 도구(UiPath), BI 도구(Tableau/Power BI)',
 'soft_skills': '커뮤니케이션 및 협업, 문제 해결 및 논리적 사고, 도메인 지식에 대한 학습 민첩성, 윤리의식 및 데이터 프라이버시 이해',
 'explain': 'AI 기술을 활용해 회계 데이터를 자동으로 수집·정리하고, 이상 거래 탐지, 예측 분석 및 자동화된 보고서를 통해 정확성과 효율성을 높이는 회계 직무. 초급 단계에서는 데이터 준비와 기본 모델링, 자동화 도구 활용에 집중.'}